In [1]:
import os
import pandas as pd
import sys  

stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf8')
sys.stdout = stdout
projects = ['abinit', 'lammps', 'mdanalysis', 'libmesh']
#projects = ['mdanalysis', 'rmg-py', 'hoomd', 'lammps', 'amber', 'xenon', 'abinit', 'libmesh']

In [2]:
df = {}
for p in projects:
    directory = p + "/"
    files = [filename for filename in os.listdir(p) if filename.endswith(".csv")]
    col_names = ["hash", 'time', 'message', 'buggy']
    p_df = pd.DataFrame(columns=col_names)
    for f in files:
        f_loc = directory + f
        temp_df = pd.read_csv(f_loc, index_col=False) 
        if p_df.shape[0] != 0:
            p_df = pd.concat([p_df, temp_df])
        else:
            p_df = temp_df
    

In [19]:
df = {}
unique_entries = {}
for p in projects:
    unique_entries[p] = {}
    directory = p + "/"
    files = [filename for filename in os.listdir(p) if filename.endswith(".csv")]
    col_names = ["hash", 'time', 'message', 'buggy']
    p_df = pd.DataFrame(columns=col_names)
    for f in files:
        f_loc = directory + f
        temp_df = pd.read_csv(f_loc, index_col=False) 
        if p_df.shape[0] != 0:
            p_df = pd.concat([p_df, temp_df])
        else:
            p_df = temp_df
    count_1 = 0
    for index, row in p_df.iterrows():
        if row['hash'] not in unique_entries[p].keys():
            unique_entries[p][row['hash']] = [row['buggy'], 0]
        else:
            count_1 += 1
            #print(row['buggy'], unique_entries[row['hash']][0])
            if row['buggy'] != unique_entries[p][row['hash']][0]:
                unique_entries[p][row['hash']][1] = 1 
    count = 0
    for key, value in unique_entries[p].items():
        #print(value)
        if value[1] == 1:
            count += 1
    print(p, p_df.shape[0], float(count_1)/p_df.shape[0], float(count)/count_1)
    p_df = p_df.drop_duplicates(subset=['hash'], keep="last").reset_index(drop=True)
    df[p] = p_df 
    print(p, p_df.shape)
    #p_df.to_csv(p + "_human.csv", index=False)
    #df[p] = p_df

('abinit', 11258, 0.5210516965713271, 0.05830207978179339)
('abinit', (5392, 4))
('lammps', 26616, 0.7248271716260896, 0.011196350818992328)
('lammps', (7324, 4))
('mdanalysis', 5517, 0.401305057096248, 0.1025293586269196)
('mdanalysis', (3303, 4))
('libmesh', 9339, 0.0706713780918728, 0.1)
('libmesh', (8679, 4))


['abinit', 'lammps', 'mdanalysis', 'libmesh']

In [15]:
for p in projects:
    print(df[p].shape)

(5392, 4)
(7324, 4)
(3303, 4)
(8679, 4)


In [ ]:
processed_df = {}
for p in projects:
    col_names = ["Document Title", 'Abstract', 'Year', 'PDF Link', 'label']
    p_df = pd.DataFrame(columns=col_names)
    temp_df = df[p] 
    p_df['Document Title'] = temp_df['hash']
    p_df['Abstract'] = temp_df['message']
    p_df['Year'] = [2018]*temp_df.shape[0]
    p_df.to_csv(p + "_fast.csv", index=False)

In [3]:
import itertools 

def most_common(L):
  groups = itertools.groupby(sorted(L))
  def _auxfun((item, iterable)):
    return len(list(iterable)), -L.index(item)
  return max(groups, key=_auxfun)[0]

In [6]:
df = {}
unique_entries = {}
for p in projects:
    unique_entries[p] = {}
    directory = p + "/"
    files = [filename for filename in os.listdir(p) if filename.endswith(".csv")]
    col_names = ["hash", 'time', 'message', 'buggy']
    p_df = pd.DataFrame(columns=col_names)
    for f in files:
        f_loc = directory + f
        temp_df = pd.read_csv(f_loc, index_col=False) 
        if p_df.shape[0] != 0:
            p_df = pd.concat([p_df, temp_df]).reset_index(drop=True)
        else:
            p_df = temp_df
    for index, row in p_df.iterrows():
        if row['hash'] not in unique_entries[p].keys():
            unique_entries[p][row['hash']] = [row['buggy']]
        else:
            #print(row['buggy'], unique_entries[row['hash']][0])
            unique_entries[p][row['hash']].append(row['buggy'])
    
    #p_df = p_df.drop_duplicates(subset=['hash'], keep="last")
    #p_df.to_csv(p + "_human.csv", index=False)
    #df[p] = p_df
    #print(p, p_df.shape)

In [ ]:
print(unique_entries['abinit'])

In [27]:
import numpy as np
import random
from sklearn.metrics import confusion_matrix
for p in projects:
    predicted = []
    actual = []
    col_names = ["hash", 'time', 'message', 'buggy']
    p_df = pd.DataFrame(columns=col_names)
    p_df_fast = pd.read_csv("/home/huyqt7/Projects/PhD/data-collection/labeled_commits/fastread/" + p + "_fast_labeled.csv")
    #print(p_df_fast.shape[0], df[p].shape[0])
    #no_p_df = p_df_fast[p_df_fast.code == "undetermined"]
    #no_labels = no_p_df.shape[0]*[0]
    #random_1_undetermined = random.sample(range(0, no_p_df.shape[0]), int(0.08 * no_p_df.shape[0]) + 1)
    #for i in random_1_undetermined:
    #    no_labels[i] = 1
    #no_p_df['label'] = no_labels
    p_df_fast = p_df_fast[p_df_fast.code != "undetermined"]
    #print(p_df_fast.shape[0])
    p_df_fast['label'] = pd.Series(np.where(p_df_fast.code == 'yes', 1, 0),
          p_df_fast.index)
    count = 0
    yes_und = 0
    actual = []
    #predicted = p_df_fast['label'].values.tolist()
    for index, row in p_df_fast.iterrows():
        predicted.append(row['label'])
        comparable_idx = list(df[p][df[p]['hash'] == row['Document Title']].index)
        if not actual:
            actual = df[p].loc[comparable_idx]['buggy'].values.tolist()
        else:
            actual.extend(df[p].loc[comparable_idx]['buggy'].values.tolist())
    tn, fp, fn, tp = confusion_matrix(actual, predicted).ravel()
    print(p, tn, fp, fn, tp)

('abinit', 388, 287, 57, 389)
('lammps', 276, 476, 9, 97)
('mdanalysis', 458, 355, 41, 420)
('libmesh', 720, 756, 102, 643)


In [33]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import string

ps = PorterStemmer()

for p in df.keys():
    p_df_fast = pd.read_csv("/home/huyqt7/Projects/PhD/data-collection/labeled_commits/fastread/" + p + "_fast_labeled.csv")
    p_df_fast = p_df_fast[p_df_fast.code != "undetermined"]
    p_df_fast['label'] = pd.Series(np.where(p_df_fast.code == 'yes', 1, 0), p_df_fast.index)
    predicted = []
    actual = []
    for index, row in p_df_fast.iterrows():
        comparable_idx = list(df[p][df[p]['hash'] == row['Document Title']].index)
        if not actual:
            actual = df[p].loc[comparable_idx]['buggy'].values.tolist()
        else:
            actual.extend(df[p].loc[comparable_idx]['buggy'].values.tolist())
        m = df[p].loc[comparable_idx]['message'].values.tolist()[0]
        try:
            words = word_tokenize(m)
            stemmed_w = [ps.stem(w) for w in words]
        except TypeError:
            print("TypeError", m)
            break
        except UnicodeDecodeError:
            print("UDError", words)
            break
        #print(stemmed_w)
        bug_key_words = set(["bug", "fix", "wrong", "error", "fail", "problem", "patch"])
        check = set(stemmed_w).intersection(bug_key_words)
        if len(check) > 0:
            predicted.append(1)
        else:
            predicted.append(0)
    tn, fp, fn, tp = confusion_matrix(actual, predicted).ravel()
    print(p, tn, fp, fn, tp)

('abinit', 395, 280, 44, 402)
('libmesh', 584, 892, 58, 687)
('lammps', 137, 615, 11, 95)
('mdanalysis', 543, 270, 31, 430)


In [4]:
"('abinit', 3821, 586, 111, 394, 4912, 3791)
('lammps', 6222, 994, 11, 97, 7324, 6466)
('mdanalysis', 2308, 515, 57, 423, 3303, 2029)
('libmesh', 6637, 1270, 126, 646, 8679, 6458)"

"('abinit', 79, 367, 185, 78, 491, 1121, 3791)
('lammps', 9, 280, 494, 5, 79, 858, 6466)
('mdanalysis', 16, 481, 524, 18, 251, 1274, 2029)
('libmesh', 27, 715, 701, 107, 698, 2221, 6458)
"
a = [2, 3, 4, 5, 6]
a[[1, 2]] = 0
print(a)

TypeError: list indices must be integers, not list

In [7]:
temp_unique_entries = {}
for p in projects:
    temp_unique_entries[p] = unique_entries[p]
    for k in temp_unique_entries[p].keys():
        temp_unique_entries[p][k] = most_common(unique_entries[p][k])

In [9]:
import numpy as np
from sklearn.metrics import confusion_matrix
for p in projects:
    predicted = []
    actual = []
        
    col_names = ["hash", 'message', 'time', 'buggy']
    p_df = pd.DataFrame(columns=col_names)
    p_df_fast = pd.read_csv("fastread/" + p + "_fast_labeled.csv")
    p_df_fast_labels = p_df_fast['label'].values
    p_df_fast = p_df_fast.drop(['label', 'Year', 'PDF Link'], axis=1)
    p_df_fast['label'] = p_df_fast_labels
    #print(p_df_fast.shape[0])
    no_p_df = p_df_fast[p_df_fast.code == "undetermined"]
    
    #no_p_df['label'] = no_p_df.shape[0]*[0]
    p_df_fast = p_df_fast[p_df_fast.code != "undetermined"]
    #print(p_df_fast.shape[0])
    p_df_fast['label'] = pd.Series(np.where(p_df_fast.code == 'yes', 1, 0),
          p_df_fast.index)
    p_df_fast = p_df_fast.drop(['code'], axis=1)
    no_p_df = no_p_df.drop(['code'], axis=1)
    #p_df_fast = pd.concat([p_df_fast, no_p_df])
    count = 0
    yes_und = 0
    p_df_index = 0
    for index, row in p_df_fast.iterrows():
        if temp_unique_entries[p][row['Document Title']] == 1 or row['label'] == 1:
            row['label'] == 1
        p_df.loc[p_df_index] = row.tolist()
        p_df_index += 1
            
    for index, row in no_p_df.iterrows():
        if temp_unique_entries[p][row['Document Title']] == 1:
            row['label'] = 1
        else:
            row['label'] = 0
        #print(row.tolist())
        p_df.loc[p_df_index] = row.tolist()
        p_df_index += 1
    p_df.to_csv(p + "_concat.csv")
    print(p, p_df.shape[0])
    print(p_df.groupby('buggy').count())


('abinit', 4912)
       hash  message  time
buggy                     
0      4174     4174  4174
1       738      738   738
('lammps', 7324)
       hash  message  time
buggy                     
0      6750     6750  6750
1       574      574   574
('mdanalysis', 3303)
       hash  message  time
buggy                     
0      2513     2513  2513
1       790      790   790
('libmesh', 8679)
       hash  message  time
buggy                     
0      7253     7250  7253
1      1426     1426  1426


In [ ]:
('abinit', 258, 1121, 0.23015165031222123)
('lammps', 501, 858, 0.583916083916084)
('mdanalysis', 472, 1274, 0.3704866562009419)
('libmesh', 395, 1001, 0.3946053946053946)

In [2]:
import numpy as np
for p in projects:
    predicted = []
    actual = []
        
    col_names = ["hash", 'message', 'time', 'buggy']
    p_df_fast = pd.DataFrame(columns=col_names)
    p_df_fast = pd.read_csv("fastread/" + p + "_fast_labeled.csv")
    p_df_fast_labels = p_df_fast['label'].values
    p_df_fast = p_df_fast[p_df_fast.code != "undetermined"]
    p_df_fast = p_df_fast.drop(['label', 'Year', 'PDF Link'], axis=1)
    p_df_fast['label'] = pd.Series(np.where(p_df_fast.code == 'yes', 1, 0),
          p_df_fast.index)
    p_df_fast = p_df_fast.drop(['code'], axis=1)
    
    p_df_fast.to_csv(p + "_concat.csv")
    print(p, p_df_fast.shape[0])
    print(p_df_fast.groupby('label').count())


('amber', 967)
       Document Title  Abstract  time
label                                
0                 158       158   158
1                 809       809   809
('hoomd', 881)
       Document Title  Abstract  time
label                                
0                 117       117   117
1                 764       764   764
('pcmsolver', 328)
       Document Title  Abstract  time
label                                
0                 134       134   134
1                 194       194   194
('rmg-py', 706)
       Document Title  Abstract  time
label                                
0                  53        53    53
1                 653       653   653
('xenon', 921)
       Document Title  Abstract  time
label                                
0                 250       250   250
1                 671       671   671


In [8]:
import random

for p in projects:
    p_df_fast = pd.read_csv(p + "_concat.csv")
    random_indexes = random.sample(range(1, p_df_fast.shape[0]), 100)
    p_sample = p_df_fast.ix[random_indexes]
    p_sample.to_csv("sanity_checks/" + p +  "_sample.csv", index_label=False)

In [38]:
import numpy as np
projects = ['mdanalysis', 'libmesh', 'abinit', 'lammps']
#projects = ['mdanalysis']
for p in projects:
    timestamps = []
    hashvalues = []
    col_names = ["hash", 'message', 'time', 'buggy']
    p_df_final = pd.DataFrame(columns=col_names)
    p_df_concat = pd.read_csv("auto/" + p + "_concat.csv")
    p_df_fast = pd.read_csv(p + "_human.csv")
    for h in p_df_concat['hash'].values.tolist():
        if len(h) > 7:
            h = h[:7]
        val = p_df_fast[p_df_fast['hash'].str.startswith(h)]
        if not val.empty:
            for i, row in val.iterrows():
                if row['hash'][-2] != ".0":            
                    timestamps.append(val['time'].values.tolist()[0])
                    hashvalues.append(val['hash'].values.tolist()[0])
        else: 
            print(h)
    p_df_final['hash'] = hashvalues
    p_df_final['time'] = timestamps
    p_df_final['message'] = p_df_concat['message']
    p_df_final['buggy'] = p_df_concat['buggy']
    p_df_final.to_csv(p + "_concat.csv")
    print(p, p_df_final.shape[0])
    print(p_df_final.groupby('buggy').count())


('mdanalysis', 3344)
       hash  message  time
buggy                     
0.0    2513     2513  2513
1.0     790      790   790
('libmesh', 8679)
       hash  message  time
buggy                     
0      7253     7250  7253
1      1426     1426  1426
('abinit', 4912)
       hash  message  time
buggy                     
0      4174     4174  4174
1       738      738   738
('lammps', 7514)
       hash  message  time
buggy                     
0.0    6750     6750  6750
1.0     574      574   574
